# Quantum Protein Folding: Quickstart Guide

This notebook demonstrates the basic usage of the QuantumFold-Advantage framework.

## Overview

We'll solve a simple HP (Hydrophobic-Polar) protein folding problem using VQE.

In [ ]:
# Install if needed
# !pip install -e ..

import numpy as np
import matplotlib.pyplot as plt
from quantum_protein_folding.models import VQEFoldingModel
from quantum_protein_folding.analysis import plot_convergence, plot_conformation_2d

## 1. Define Protein Sequence

We use a simple HP model sequence where:
- H = Hydrophobic residue (prefers to be buried)
- P = Polar residue (prefers to be exposed)

Goal: Find conformation that maximizes H-H contacts.

In [ ]:
# Classic benchmark sequence
sequence = "HPHPPHHPHH"

print(f"Sequence: {sequence}")
print(f"Length: {len(sequence)}")

## 2. Initialize VQE Model

Configure the VQE solver with:
- 2D square lattice
- Hardware-efficient ansatz
- COBYLA optimizer

In [ ]:
model = VQEFoldingModel(
    sequence=sequence,
    lattice_dim=2,
    encoding_type='turn_direction',
    ansatz_type='hardware_efficient',
    ansatz_depth=3,
    optimizer='COBYLA',
    shots=1024
)

print(f"Number of qubits: {model.encoding.n_qubits}")
print(f"Number of parameters: {model.solver.n_params}")

## 3. Run VQE Optimization

This will take a few minutes depending on your hardware.

In [ ]:
# Run optimization
result = model.run(maxiter=100)

print(f"\nOptimization Results:")
print(f"Optimal Energy: {result.optimal_value:.4f}")
print(f"Iterations: {result.n_iterations}")
print(f"Best Bitstring: {result.optimal_bitstring}")

## 4. Visualize Convergence

In [ ]:
plot_convergence(
    result.convergence_history,
    title="VQE Optimization Convergence",
    ylabel="Energy"
)

## 5. Decode and Visualize Structure

In [ ]:
# Decode bitstring to conformation
conformation = model.decode_conformation(result.optimal_bitstring)

print(f"Conformation coordinates:\n{conformation}")

# Check validity
is_valid = model.validate_conformation(conformation)
print(f"\nValid conformation: {is_valid}")

In [ ]:
# Plot 2D structure
plot_conformation_2d(
    conformation,
    sequence=sequence,
    title=f"Optimal Conformation (E = {result.optimal_value:.2f})"
)

## 6. Compare with Classical Baseline

In [ ]:
from quantum_protein_folding.classical import simulated_annealing_fold

# Run simulated annealing
classical_result = simulated_annealing_fold(
    model.encoding,
    max_iterations=5000,
    seed=42
)

print(f"Classical Energy: {classical_result.energy:.4f}")
print(f"Quantum Energy: {result.optimal_value:.4f}")
print(f"Energy Gap: {result.optimal_value - classical_result.energy:.4f}")

## Summary

You've successfully:
1. ✅ Loaded an HP protein sequence
2. ✅ Configured and ran VQE optimization
3. ✅ Visualized convergence and structure
4. ✅ Compared with classical baseline

Next steps:
- Try different sequences
- Experiment with ansatz depth
- Use QAOA instead of VQE
- Run on real quantum hardware